In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

In [2]:
folder_path = '1. data_nasdaq_csv'

In [3]:
dataframes = []
company_names = []

In [4]:
# big market cap
stocks = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'ASML', 'COST', 'CSCO', 'ADBE', 'CMCSA',
          'TXN', 'AMGN', 'NFLX', 'SBUX', 'INTC', 'MDLZ', 'QCOM', 'AMAT', 'FAST', 
          'CHTR', 'ORLY', 'SNPS', 'CTAS', 'DXCM', 'BIDU', 'EA', 'SBAC', 'MAR',
          'CTSH', 'VRSK', 'TSCO']

In [6]:
paths = []
for stock in stocks:
    path = os.path.join(folder_path, stock + '.csv')
    paths.append(path)
paths

['data_nasdaq_csv/AAPL.csv',
 'data_nasdaq_csv/MSFT.csv',
 'data_nasdaq_csv/AMZN.csv',
 'data_nasdaq_csv/NVDA.csv',
 'data_nasdaq_csv/ASML.csv',
 'data_nasdaq_csv/COST.csv',
 'data_nasdaq_csv/CSCO.csv',
 'data_nasdaq_csv/ADBE.csv',
 'data_nasdaq_csv/CMCSA.csv',
 'data_nasdaq_csv/TXN.csv',
 'data_nasdaq_csv/AMGN.csv',
 'data_nasdaq_csv/NFLX.csv',
 'data_nasdaq_csv/SBUX.csv',
 'data_nasdaq_csv/INTC.csv',
 'data_nasdaq_csv/MDLZ.csv',
 'data_nasdaq_csv/QCOM.csv',
 'data_nasdaq_csv/AMAT.csv',
 'data_nasdaq_csv/FAST.csv',
 'data_nasdaq_csv/CHTR.csv',
 'data_nasdaq_csv/ORLY.csv',
 'data_nasdaq_csv/SNPS.csv',
 'data_nasdaq_csv/CTAS.csv',
 'data_nasdaq_csv/DXCM.csv',
 'data_nasdaq_csv/BIDU.csv',
 'data_nasdaq_csv/EA.csv',
 'data_nasdaq_csv/SBAC.csv',
 'data_nasdaq_csv/MAR.csv',
 'data_nasdaq_csv/CTSH.csv',
 'data_nasdaq_csv/VRSK.csv',
 'data_nasdaq_csv/TSCO.csv']

In [7]:
from sklearn.preprocessing import MinMaxScaler

window_size = 30
X_data, y_data = [], []
scaler = MinMaxScaler(feature_range=(0,1))

for path in paths:
    df = pd.read_csv(path)
    df1 = df.reset_index()['Close']
    df1 = scaler.fit_transform(np.array(df1).reshape(-1,1))
    for i in range(len(df1)-window_size-7):
        a = df1[i:(i + window_size)]
        X_data.append(a)
        b = df1[(i + window_size):(i + window_size+7)] 
        y_data.append(b) 

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

In [9]:
### Split the data into training, validation and test set
from helper_functions import data_preprocessing as pp
X_train, y_train, X_val, y_val, X_test, y_test = pp.split_dataset(X_data, y_data, 0.1)

print("Shape of training set: ", X_train.shape)
print("Shape of validation set: ", X_val.shape)
print("Shape of test set: ", X_test.shape)

Shape of training set:  (185690, 30, 1)
Shape of validation set:  (20633, 30, 1)
Shape of test set:  (22925, 30, 1)


In [10]:
# Reshape the numpy array to fit the neural network input shape requirement
X_train = X_train.reshape(X_train.shape[0], window_size, 1)
X_val = X_val.reshape(X_val.shape[0], window_size, 1)
X_test = X_test.reshape(X_test.shape[0], window_size, 1)

# Print shape of the training, validation and test set
print("Shape of training set: ", X_train.shape)
print("Shape of validation set: ", X_val.shape)
print("Shape of test set: ", X_test.shape)

Shape of training set:  (185690, 30, 1)
Shape of validation set:  (20633, 30, 1)
Shape of test set:  (22925, 30, 1)


In [11]:
### Reshape the data to fit the LSTM model

In [12]:
### Create and train the model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Define the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(30, 1), activation='relu'))  # LSTM layer with 128 units
model.add(LSTM(50, activation='relu', return_sequences=True))  
model.add(LSTM(50, activation='relu'))  
model.add(Dense(7))  # Dense output layer with 7 units

early_stopping = EarlyStopping(monitor='val_mse', patience=10, restore_best_weights=True)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 128)           66560     
                                                                 
 lstm_1 (LSTM)               (None, 30, 50)            35800     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 7)                 357       
                                                                 
Total params: 122,917
Trainable params: 122,917
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile and train the model with Mean Squared Error loss function
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mse'])

In [14]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=20,batch_size=1028,verbose=1,callbacks = [early_stopping])

Epoch 1/20
181/181 [==============================] - 46s 224ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 9.1998e-04 - val_mse: 9.1998e-04
Epoch 2/20
181/181 [==============================] - 31s 172ms/step - loss: 3.9491e-04 - mse: 3.9491e-04 - val_loss: 6.9110e-04 - val_mse: 6.9110e-04
Epoch 3/20
181/181 [==============================] - 35s 191ms/step - loss: 3.2189e-04 - mse: 3.2189e-04 - val_loss: 5.2305e-04 - val_mse: 5.2305e-04
Epoch 4/20
181/181 [==============================] - 32s 178ms/step - loss: 2.5762e-04 - mse: 2.5762e-04 - val_loss: 5.5070e-04 - val_mse: 5.5070e-04
Epoch 5/20
181/181 [==============================] - 33s 185ms/step - loss: 2.2749e-04 - mse: 2.2749e-04 - val_loss: 4.0581e-04 - val_mse: 4.0581e-04
Epoch 6/20
181/181 [==============================] - 32s 176ms/step - loss: 1.9334e-04 - mse: 1.9334e-04 - val_loss: 4.0139e-04 - val_mse: 4.0139e-04
Epoch 7/20
181/181 [==============================] - 31s 174ms/step - loss: 1.7603e-04 - mse: 1.7603e

In [15]:
### Get prediction on the test data and convert the result back to stock price (i.e., de-normalization)
from sklearn.metrics import mean_squared_error

# Get prediction on the test data
y_pred = model.predict(X_test)
y_pred = np.reshape(y_pred, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))
print("MSE on the test set: ", mean_squared_error(y_pred, y_test))

717/717 [==============================] - 12s 16ms/step
MSE on the test set:  0.00022199837584700038


In [19]:
# Try another model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

# Define the LSTM model
model2 = Sequential()
model2.add(LSTM(128, return_sequences=True, input_shape=(30, 1), activation='relu'))  # LSTM layer with 128 units
model2.add(Dropout(0.2))  # Dropout layer for regularization
model2.add(LSTM(64, activation='relu'))  # LSTM layer with 64 units
model2.add(Dense(32, activation='relu'))  # Dense layer with 32 units and ReLU activation
model2.add(Dense(1))  # Dense output layer with 7 units

# Compile and train the model with Mean Squared Error loss function
model2.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mse'])
model2.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=20,batch_size=1028,verbose=1,callbacks = [early_stopping])

Epoch 1/20
181/181 [==============================] - 27s 130ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 7.6127e-04 - val_mse: 7.6127e-04
Epoch 2/20
181/181 [==============================] - 22s 121ms/step - loss: 4.6427e-04 - mse: 4.6427e-04 - val_loss: 7.6889e-04 - val_mse: 7.6889e-04
Epoch 3/20
181/181 [==============================] - 23s 128ms/step - loss: 3.7416e-04 - mse: 3.7416e-04 - val_loss: 7.3178e-04 - val_mse: 7.3178e-04
Epoch 4/20
181/181 [==============================] - 23s 126ms/step - loss: 3.1505e-04 - mse: 3.1505e-04 - val_loss: 8.5921e-04 - val_mse: 8.5921e-04
Epoch 5/20
181/181 [==============================] - 21s 117ms/step - loss: 2.7889e-04 - mse: 2.7889e-04 - val_loss: 0.0011 - val_mse: 0.0011
Epoch 6/20
181/181 [==============================] - 23s 127ms/step - loss: 2.4705e-04 - mse: 2.4705e-04 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 7/20
181/181 [==============================] - 23s 125ms/step - loss: 2.3778e-04 - mse: 2.3778e-04 - val_loss: 